<a href="https://colab.research.google.com/github/rsglick/drl/blob/master/notebooks/LLCv2_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Ray

Ray is a fast and simple framework for building and running distributed applications.

https://github.com/ray-project/ray

In [0]:
print('NOTE: Intentionally crashing session to use the newly installed library.\n')

!apt-get install -y xvfb x11-utils python-opengl swig cmake ffmpeg freeglut3-dev

!pip uninstall -y pyarrow
!pip install ray[debug] ray[tune] ray[rllib] dm-tree lz4 gputil
!pip install bs4 

# Install Other Gym Environments
!pip install Box2D box2d-py box2d-kengz gym[box2d] gym[Box_2D]


# A hack to force the runtime to restart, needed to include the above dependencies.
import os
os._exit(0)

NOTE: Intentionally crashing session to use the newly installed library.

Reading package lists... Done
Building dependency tree       
Reading state information... Done
freeglut3-dev is already the newest version (2.8.1-3).
freeglut3-dev set to manually installed.
cmake is already the newest version (3.10.2-1ubuntu2.18.04.1).
ffmpeg is already the newest version (7:3.4.6-0ubuntu0.18.04.1).
The following additional packages will be installed:
  libxxf86dga1 swig3.0
Suggested packages:
  libgle3 swig-doc swig-examples swig3.0-examples swig3.0-doc mesa-utils
The following NEW packages will be installed:
  libxxf86dga1 python-opengl swig swig3.0 x11-utils xvfb
0 upgraded, 6 newly installed, 0 to remove and 31 not upgraded.
Need to get 2,590 kB of archives.
After this operation, 14.2 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libxxf86dga1 amd64 2:1.1.4-1 [13.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 python-ope

In [0]:
%matplotlib inline

In [28]:
import os
import matplotlib.pyplot as plt
import numpy as np
import random
from typing import Dict

import torch

import gym
import ray

from ray.tune.logger import pretty_print
from ray import tune

from ray.rllib.agents import ppo, sac, ddpg
from ray.rllib.agents.callbacks import DefaultCallbacks
from ray.rllib.evaluation import MultiAgentEpisode, RolloutWorker
from ray.rllib.env import BaseEnv
from ray.rllib.policy import Policy

ray.init(ignore_reinit_error=True, log_to_driver=False)

2020-05-29 17:13:47,828	ERROR worker.py:700 -- Calling ray.init() again after it has already been called.


# Create Continous Gym Environment




In [0]:
# LunarLanderContinuous-v2
# MountainCarContinuous-v0
# Pendulum-v0

env_name = "LunarLanderContinuous-v2"
#env = gym.make(env_name)

#log_dir = f"./gym/{env_name}"
#os.makedirs(log_dir, exist_ok=True)

In [0]:
class myCallback(DefaultCallbacks):
    def on_episode_end(self, worker: RolloutWorker, base_env: BaseEnv,
                       policies: Dict[str, Policy], episode: MultiAgentEpisode,
                       **kwargs):
        #if episode.episode_id % 10 == 0:
        print(episode)
        #print(f"(Ep{episode.episode_id}) Reward Mean: {episode.episode_reward_mean}")  

    def on_train_result(self, trainer, result: dict, **kwargs):
        print("trainer.train() result: {} -> {} episodes".format(
            trainer, result["episodes_this_iter"]))
        
        
config = sac.DEFAULT_CONFIG.copy()
#config["framework"] = "torch"
config["use_pytorch"] = True
config["train_batch_size"] = 256 #tune.grid_search([100, 256, 1000])
config["num_gpus"]=0 #1 
config["env"]=env_name
config["callbacks"]=myCallback

stop = {}
stop["training_iteration"] = 10
stop["episode_reward_mean"] = 200 #200 reward is target mean for Lunar Lander 



In [0]:
# agent = sac.SACTrainer(config=config)

# for i in range(10):
#     result = agent.train()
#     print(pretty_print(result))

# checkpoint_path = agent.save()
# print(checkpoint_path)

In [37]:
sac_analysis = tune.run("SAC", verbose=1, config=config, stop=stop)



Trial name,status,loc,iter,total time (s),ts,reward
SAC_LunarLanderContinuous-v2_00000,TERMINATED,,10,19.811,1807,-246.78


In [0]:
os.system(f"tar -cvzf /root/ray_results.tar.gz /root/ray_results")

In [0]:
import time
time.sleep(60)

In [0]:
from google.colab import files
files.download('/root/ray_results.tar.gz')

In [0]:
%load_ext tensorboard

In [0]:
%tensorboard --logdir ~/ray_results/SAC